# Demonstration of control of SBIG CCD cameras

In [1]:
# Load the PANOPTES module dir
import sys
sys.path.append('../../')

%load_ext autoreload
%autoreload 2

In [2]:
from pocs.camera.sbig import Camera

In [3]:
from pocs.camera.sbigudrv import SBIGDriver

In [4]:
sd = SBIGDriver()

2017-01-06 19:48:18 aaomc48as.aao.gov.au panoptes[93741] INFO Found 2 SBIG cameras


In [5]:
del sd

In [6]:
sd2 = SBIGDriver()

2017-01-06 19:48:20 aaomc48as.aao.gov.au panoptes[93741] INFO Found 0 SBIG cameras


In [7]:
del sd2

In [8]:
c0 = Camera(set_point=0)
c1 = Camera(set_point=0)

2017-01-06 19:33:03 aaomc48as.aao.gov.au panoptes[93516] INFO Found 0 SBIG cameras
2017-01-06 19:33:03 aaomc48as.aao.gov.au panoptes[93516] ERROR No connected SBIG cameras available!
2017-01-06 19:33:03 aaomc48as.aao.gov.au panoptes[93516] WARNING Could not connect to SBIG Camera!
2017-01-06 19:33:03 aaomc48as.aao.gov.au panoptes[93516] ERROR No connected SBIG cameras available!
2017-01-06 19:33:03 aaomc48as.aao.gov.au panoptes[93516] WARNING Could not connect to SBIG Camera!


In [6]:
c0.uid

'83F010771'

In [7]:
c1.uid

'83F010801'

In [6]:
c0._info

{'AO8': False,
 'Biorad_TDI_support': False,
 'STXL': False,
 'Truesense': False,
 'bad_columns': [],
 'camera_name': 'SBIG STF-8300 CCD Camera',
 'camera_type': 'STF_CAMERA',
 'colour': False,
 'dump_extra': 0,
 'electronic_shutter': False,
 'firmware_version': '2.22',
 'frame_buffer': True,
 'frame_transfer': False,
 'imaging_ABG': True,
 'mechanical_shutter': True,
 'readout_modes': {'RM_1X1': {'gain': <Quantity 0.38 electron / adu>,
   'height': <Quantity 2532.0 pix>,
   'pixel_height': <Quantity 5.4 um>,
   'pixel_width': <Quantity 5.4 um>,
   'width': <Quantity 3352.0 pix>},
  'RM_1X1_VOFFCHIP': {'gain': <Quantity 0.38 electron / adu>,
   'height': <Quantity 2532.0 pix>,
   'pixel_height': <Quantity 5.4 um>,
   'pixel_width': <Quantity 5.4 um>,
   'width': <Quantity 3352.0 pix>},
  'RM_2X2': {'gain': <Quantity 0.38 electron / adu>,
   'height': <Quantity 1266.0 pix>,
   'pixel_height': <Quantity 10.8 um>,
   'pixel_width': <Quantity 10.8 um>,
   'width': <Quantity 1676.0 pix>},
 

In [7]:
c1._info

{'AO8': False,
 'Biorad_TDI_support': False,
 'STXL': False,
 'Truesense': False,
 'bad_columns': [],
 'camera_name': 'SBIG STF-8300 CCD Camera',
 'camera_type': 'STF_CAMERA',
 'colour': False,
 'dump_extra': 0,
 'electronic_shutter': False,
 'firmware_version': '2.22',
 'frame_buffer': True,
 'frame_transfer': False,
 'imaging_ABG': True,
 'mechanical_shutter': True,
 'readout_modes': {'RM_1X1': {'gain': <Quantity 0.37 electron / adu>,
   'height': <Quantity 2532.0 pix>,
   'pixel_height': <Quantity 5.4 um>,
   'pixel_width': <Quantity 5.4 um>,
   'width': <Quantity 3352.0 pix>},
  'RM_1X1_VOFFCHIP': {'gain': <Quantity 0.37 electron / adu>,
   'height': <Quantity 2532.0 pix>,
   'pixel_height': <Quantity 5.4 um>,
   'pixel_width': <Quantity 5.4 um>,
   'width': <Quantity 3352.0 pix>},
  'RM_2X2': {'gain': <Quantity 0.37 electron / adu>,
   'height': <Quantity 1266.0 pix>,
   'pixel_height': <Quantity 10.8 um>,
   'pixel_width': <Quantity 10.8 um>,
   'width': <Quantity 1676.0 pix>},
 

In [8]:
print(c0.CCD_cooling_enabled, c1.CCD_cooling_enabled)

True True


In [9]:
print(c0.CCD_set_point, c1.CCD_set_point)

0.18901872786513252 deg_C 0.18901872786513252 deg_C


In [10]:
print(c0.CCD_temp, c1.CCD_temp)

-0.23181540522300637 deg_C 0.18901872786513252 deg_C


In [11]:
print(c0.CCD_cooling_power, c1.CCD_cooling_power)

48.23529411764706 47.450980392156865


In [12]:
e0 = c0.take_exposure(1.0, filename='test0.fits')
e1 = c1.take_exposure(1.0, filename='test1.fits', dark = True)

In [13]:
for i in range(2,5):
    c0.take_exposure(0, filename='bias{}.fits'.format(i), dark=True, blocking=True)

In [14]:
from ctypes.util import find_library

In [18]:
find_library('sbigudrv')

'/Library/Frameworks/sbigudrv.framework/sbigudrv'

In [19]:
import ctypes

In [25]:
ctypes.CDLL(find_library('SBIGUDRV'))

<CDLL '/Library/Frameworks/SBIGUDRV.framework/SBIGUDRV', handle 103ac4230 at 0x1120967b8>

In [28]:
ctypes.CDLL(find_library('sbigudrv'))

<CDLL '/Library/Frameworks/sbigudrv.framework/sbigudrv', handle 103ac4230 at 0x111498a90>

In [29]:
ctypes.cdll.LoadLibrary(find_library('SBIGUDRV'))

<CDLL '/Library/Frameworks/SBIGUDRV.framework/SBIGUDRV', handle 103ac4230 at 0x1114984a8>